**to be excetued in google colab**

In [ ]:
!pip install --upgrade pandas
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

path = "INSERT GDRIVE PATH HERE"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
from collections import defaultdict

In [ ]:
import torch
from transformers import BertModel, BertTokenizer

# Load pre-trained BERT model
model = BertModel.from_pretrained("bert-base-german-cased")
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

# Define a function to generate sentence embeddings
def get_sentence_embeddings(sentence):
    #torch.cpu.empty_cache()
    input_ids = torch.tensor([tokenizer.encode(sentence, max_length=512, truncation=True)])
    if torch.cuda.is_available():
        input_ids = input_ids.to('cuda')
        model.to('cuda')
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]
    last_hidden_states = last_hidden_states.cpu().numpy()
    return last_hidden_states[0][0]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
nrw_social = [json.loads(line) for line in open(f"{path}fediverse-nrw.social-toots.log", "r", encoding="utf-8")]
sueden_social = [json.loads(line) for line in open(f"{path}fediverse-sueden.social-toots.log", "r", encoding="utf-8")]
troet_cafe = [json.loads(line) for line in open(f"{path}fediverse-troet.cafe-toots.log", "r", encoding="utf-8")]

In [ ]:
troet_cafe = pd.DataFrame(troet_cafe)
sueden_social = pd.DataFrame(sueden_social)
nrw_social = pd.DataFrame(nrw_social)

In [ ]:
#delete accounts with less than k toots

user_no_tweets = defaultdict(int)
for x in range(len(troet_cafe)):
  user = troet_cafe.at[x, "account"]
  troet_cafe.at[x, "user_id"] = str(user["id"])
  user_no_tweets[str(user["id"])] += 1
troet_cafe["no_toots"] = troet_cafe["user_id"].apply(lambda x: user_no_tweets.get(x))

user_no_tweets = defaultdict(int)
for x in range(len(sueden_social)):
  user = sueden_social.at[x, "account"]
  sueden_social.at[x, "user_id"] = str(user["id"])
  user_no_tweets[str(user["id"])] += 1
sueden_social["no_toots"] = sueden_social["user_id"].apply(lambda x: user_no_tweets.get(x))

user_no_tweets = defaultdict(int)
for x in range(len(nrw_social)):
  user = nrw_social.at[x, "account"]
  nrw_social.at[x, "user_id"] = str(user["id"])
  user_no_tweets[str(user["id"])] += 1
nrw_social["no_toots"] = nrw_social["user_id"].apply(lambda x: user_no_tweets.get(x))


troet_cafe = troet_cafe.query("no_toots > 4").reset_index()
sueden_social = sueden_social.query("no_toots > 4").reset_index()
nrw_social = nrw_social.query("no_toots > 4").reset_index()

In [ ]:
# apply get_text to all toots
troet_cafe["text"] = troet_cafe["content"].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
sueden_social["text"] = sueden_social["content"].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
nrw_social["text"] = nrw_social["content"].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())

<ipython-input-9-42bb84433a97>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  troet_cafe["text"] = troet_cafe["content"].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
<ipython-input-9-42bb84433a97>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  sueden_social["text"] = sueden_social["content"].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
<ipython-input-9-42bb84433a97>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  nrw_social["text"] = nrw_social["content"].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())


In [ ]:
troet_cafe["embedding"] = troet_cafe["text"].apply(lambda x: get_sentence_embeddings(x))

sueden_social["embedding"] = sueden_social["text"].apply(lambda x: get_sentence_embeddings(x))

nrw_social["embedding"] = nrw_social["text"].apply(lambda x: get_sentence_embeddings(x))